# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [4]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report,accuracy_score,precision_score,recall_score,f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

[nltk_data] Downloading package punkt to /Users/leec/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/leec/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/leec/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [5]:
engine = create_engine('sqlite:///figure8_cleandata.db')
df = pd.read_sql_table("clean_table",con=engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# load data from database
def load_data():
    engine = create_engine('sqlite:///figure8_cleandata.db')
    df = pd.read_sql_table("clean_table",con=engine)
    category_names = df.columns[4:]

    X = df[['message']].values[:, 0]
    Y = df[category_names].values
    Y = Y.astype("int")
    return X, Y,category_names

In [11]:
engine = create_engine('sqlite:///figure8_cleandata.db')
df = pd.read_sql_table("clean_table",con=engine)
category_names = df.columns[4:]

X = df['message'].values
Y = df[category_names].values
Y = Y.astype("int")

In [15]:
df['message'].values

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name', ...,
       "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families.",
       'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.',
       'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.'],
      dtype=object)

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
def build_model():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(), n_jobs=-1))#The number of jobs to use for the computation. -1 means using all processors.
    ])

    parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4],
        'features__transformer_weights': (
            {'text_pipeline': 1, 'starting_verb': 0.5},
            {'text_pipeline': 0.5, 'starting_verb': 1},
            {'text_pipeline': 0.8, 'starting_verb': 1},
        )
    }

    cv = GridSearchCV(pipeline, param_grid=parameters)

    return cv

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
# load data
X, y = load_data()

# perform train test split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [6]:
pipeline = Pipeline([
('vect', CountVectorizer(tokenizer=tokenize)),
('tfidf', TfidfTransformer()),
('clf', MultiOutputClassifier(RandomForestClassifier()))#The number of jobs to use for the computation.
])

In [7]:
# train classifier
pipeline.fit(X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
cat_cols = df.iloc[:,4:].columns
def multioutput_classification_report(y_test, y_pred):
    """ Function for multioutput classification report 
    INPUT: y_test, y_pred
    OUTPUT: Precision Score, Recall Score, F1 Score and Accuracy
    """
    for i in range(0, len(cat_cols)):
        print(cat_cols[i])
        print("\t\t\t Precision: {:.4f}\t Recall: {:.4f}\t F1_score: {:.4f}\t Accuracy: {:.4f}\t".format(
            precision_score(y_test[:, i], y_pred[:, i], average='weighted'),
            recall_score(y_test[:, i], y_pred[:, i], average='weighted'),
            f1_score(y_test[:, i], y_pred[:, i], average='weighted'),
            accuracy_score(y_test[:, i], y_pred[:, i])
        ))


In [9]:
multioutput_classification_report(y_test, y_pred)

related
			 Precision: 0.7935	 Recall: 0.8055	 F1_score: 0.7702	 Accuracy: 0.8055	
request
			 Precision: 0.8908	 Recall: 0.8906	 F1_score: 0.8735	 Accuracy: 0.8906	
offer
			 Precision: 0.9930	 Recall: 0.9965	 F1_score: 0.9947	 Accuracy: 0.9965	
aid_related
			 Precision: 0.7809	 Recall: 0.7768	 F1_score: 0.7699	 Accuracy: 0.7768	
medical_help
			 Precision: 0.8963	 Recall: 0.9200	 F1_score: 0.8876	 Accuracy: 0.9200	
medical_products
			 Precision: 0.9424	 Recall: 0.9518	 F1_score: 0.9315	 Accuracy: 0.9518	
search_and_rescue
			 Precision: 0.9626	 Recall: 0.9712	 F1_score: 0.9579	 Accuracy: 0.9712	
security
			 Precision: 0.9660	 Recall: 0.9828	 F1_score: 0.9743	 Accuracy: 0.9828	
military
			 Precision: 0.9664	 Recall: 0.9678	 F1_score: 0.9538	 Accuracy: 0.9678	
child_alone
			 Precision: 1.0000	 Recall: 1.0000	 F1_score: 1.0000	 Accuracy: 1.0000	
water
			 Precision: 0.9517	 Recall: 0.9531	 F1_score: 0.9391	 Accuracy: 0.9531	
food
			 Precision: 0.9196	 Recall: 0.9231	 F1_score: 0.9

/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
pipeline.get_params().keys()
parameters = {
    'clf__estimator__n_estimators': [20, 50],
    'clf__estimator__min_samples_split': [2, 4],
    }

cv = GridSearchCV(pipeline, param_grid=parameters)

cv.fit(X_train, y_train)

In [12]:
cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7fc7547619e0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [20, 50]})

In [25]:
genre_counts = df.groupby('genre').count()['message']
genre_names = list(genre_counts.index)
genre_names

['direct', 'news', 'social']

In [72]:
cat_counts=df.groupby('genre').mean()["medical_help"]
cat_names = list(cat_counts.index)
cat_counts2=df.groupby('genre').mean()["medical_products"]
cat_names2 = list(cat_counts2.index)

In [73]:
import plotly.graph_objects as go

fig = go.Figure([go.Bar(name = 'medical_help',x=cat_names, y=cat_counts),
               go.Bar(name='medical_products', x=cat_names2, y=cat_counts2)])
fig.show()

In [56]:
df.groupby('genre').mean()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
genre,,,,,,,,,,,,,,,,,,,,,
direct,6223.009242,0.690441,0.343073,0.004201,0.402539,0.054985,0.043876,0.020071,0.012229,0.004201,...,0.007282,0.017084,0.141150,0.028099,0.029126,0.003827,0.074122,0.005881,0.019231,0.335418
news,22969.259242,0.819845,0.046038,0.004996,0.448390,0.108293,0.060718,0.033894,0.022443,0.061410,...,0.016755,0.066482,0.327415,0.133272,0.110522,0.017293,0.069787,0.031666,0.080701,0.065176
social,13483.058996,0.887866,0.072385,0.002929,0.275732,0.032218,0.020502,0.028033,0.020084,0.005439,...,0.005439,0.041423,0.624686,0.043515,0.285356,0.006695,0.312552,0.021757,0.048954,0.254393


In [70]:
genre_mean_news[genre_news_name]

,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
genre,,,,,,,,,,,,,,,,,,,,,
news,0.44839,0.108293,0.060718,0.033894,0.022443,0.06141,0.0,0.060564,0.077089,0.085082,...,0.016755,0.066482,0.327415,0.133272,0.110522,0.017293,0.069787,0.031666,0.080701,0.065176


In [71]:
genre_mean = df.groupby('genre').mean().iloc[:,4:]
genre_mean_news = genre_mean.query("genre == 'news'")
genre_news_name = list(genre_mean_news.columns)
fig = go.Figure([go.Bar(name = 'news ',x=genre_news_name, y=genre_mean_news[genre_news_name])])
fig.show()

In [77]:
top_cat_count = df.iloc[:,4:].mean().sort_values(ascending=False)[1:20]
top_cat_names = list(top_cat_count.index)
fig = go.Figure([go.Bar(name = 'Top 20 ',x=top_cat_names, y=top_cat_count)])
fig.show()

In [ ]:
top_cat_count = df.iloc[:,4:].mean().sort_values(ascending=True)[1:30]
top_cat_names = list(top_cat_count.index)
aid_counts = df.groupby('aid_related').count()['message']
aid_names = list(aid_counts.index)
cat_related_names = ['aid_related', 'weather_related', 'infrastructure_related']
    

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
y_pred = cv.predict(X_test)
multioutput_classification_report(y_test, y_pred)

related
			 Precision: 0.7950	 Recall: 0.8069	 F1_score: 0.7728	 Accuracy: 0.8069	
request
			 Precision: 0.8906	 Recall: 0.8896	 F1_score: 0.8714	 Accuracy: 0.8896	
offer
			 Precision: 0.9930	 Recall: 0.9965	 F1_score: 0.9947	 Accuracy: 0.9965	
aid_related
			 Precision: 0.7698	 Recall: 0.7666	 F1_score: 0.7593	 Accuracy: 0.7666	
medical_help
			 Precision: 0.8892	 Recall: 0.9190	 F1_score: 0.8844	 Accuracy: 0.9190	
medical_products
			 Precision: 0.9436	 Recall: 0.9521	 F1_score: 0.9322	 Accuracy: 0.9521	
search_and_rescue
			 Precision: 0.9594	 Recall: 0.9711	 F1_score: 0.9584	 Accuracy: 0.9711	
security
			 Precision: 0.9660	 Recall: 0.9828	 F1_score: 0.9743	 Accuracy: 0.9828	
military
			 Precision: 0.9631	 Recall: 0.9674	 F1_score: 0.9530	 Accuracy: 0.9674	
child_alone
			 Precision: 1.0000	 Recall: 1.0000	 F1_score: 1.0000	 Accuracy: 1.0000	
water
			 Precision: 0.9482	 Recall: 0.9524	 F1_score: 0.9388	 Accuracy: 0.9524	
food
			 Precision: 0.9266	 Recall: 0.9300	 F1_score: 0.9

/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [2]:
from sklearn.svm import SVC
#Improve the pipeline
pipeline_svc = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(SVC()))
])


NameError: name 'Pipeline' is not defined

In [15]:
parameters_svc = {
    'clf__estimator__C': [0.01, 0.1, 1.0],
    'clf__estimator__kernel': ['rbf', 'poly'],
    'clf__estimator__gamma': [0.01, 0.1, 1.0],

    }

cv_svc = GridSearchCV(pipeline_svc, param_grid=parameters_svc)

In [1]:
pipeline_svc.get_params().keys()

NameError: name 'pipeline_svc' is not defined

In [ ]:
# train classifier
cv_svc.fit(X_train, y_train)

# predict on test data
y_pred_svc = cv_svc.predict(X_test)

# reports
multioutput_classification_report(y_test, y_pred_svc)

/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 334, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/multioutput.py", line 350, in fit
    super().fit(X, Y, sample_weight, **fit_params)
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/multioutput.py", line 175, in fit
    for i in range(y.shape[1]))
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/joblib/parallel.py", line 924, in __call_

/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 334, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/multioutput.py", line 350, in fit
    super().fit(X, Y, sample_weight, **fit_params)
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/multioutput.py", line 175, in fit
    for i in range(y.shape[1]))
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/joblib/parallel.py", line 924, in __call_

/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 334, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/multioutput.py", line 350, in fit
    super().fit(X, Y, sample_weight, **fit_params)
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/multioutput.py", line 175, in fit
    for i in range(y.shape[1]))
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/joblib/parallel.py", line 924, in __call_

/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 334, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/multioutput.py", line 350, in fit
    super().fit(X, Y, sample_weight, **fit_params)
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/multioutput.py", line 175, in fit
    for i in range(y.shape[1]))
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/joblib/parallel.py", line 924, in __call_

/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 334, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/multioutput.py", line 350, in fit
    super().fit(X, Y, sample_weight, **fit_params)
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/multioutput.py", line 175, in fit
    for i in range(y.shape[1]))
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/joblib/parallel.py", line 924, in __call_

...

/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 334, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/multioutput.py", line 350, in fit
    super().fit(X, Y, sample_weight, **fit_params)
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/multioutput.py", line 175, in fit
    for i in range(y.shape[1]))
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/joblib/parallel.py", line 924, in __call_

/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 334, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/multioutput.py", line 350, in fit
    super().fit(X, Y, sample_weight, **fit_params)
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/multioutput.py", line 175, in fit
    for i in range(y.shape[1]))
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/joblib/parallel.py", line 924, in __call_

/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 334, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/multioutput.py", line 350, in fit
    super().fit(X, Y, sample_weight, **fit_params)
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/multioutput.py", line 175, in fit
    for i in range(y.shape[1]))
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/joblib/parallel.py", line 924, in __call_

/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 334, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/multioutput.py", line 350, in fit
    super().fit(X, Y, sample_weight, **fit_params)
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/multioutput.py", line 175, in fit
    for i in range(y.shape[1]))
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/joblib/parallel.py", line 924, in __call_

/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 334, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/multioutput.py", line 350, in fit
    super().fit(X, Y, sample_weight, **fit_params)
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/sklearn/multioutput.py", line 175, in fit
    for i in range(y.shape[1]))
  File "/Users/leec/opt/anaconda3/lib/python3.7/site-packages/joblib/parallel.py", line 924, in __call_

### 9. Export your model as a pickle file

In [ ]:
with open('classifier.pkl', 'wb') as file:
    pickle.dump(cv, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.